<a href="https://colab.research.google.com/github/MuayThaiLegz/PracticeCrazy/blob/main/WorkingVoiceTranslationOpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-machine-learning-interpretability-aml?view=azureml-api-1

In [1]:
#!pip install datasets
!pip install azureml-interpret
!pip install gTTS
!pip install translate


#!pip install openai
#!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#!pip install shap

#!pip install pycocotools

#!pip install pytesseract pillow
#!pip install datasets


In [4]:
#from datasets import load_dataset

# Load the Conceptual Captions dataset
#conceptual_captions_dataset = load_dataset('conceptual_captions')


In [5]:
#conceptual_captions_train_data = conceptual_captions_dataset['train']


In [14]:
import IPython as ip
from scipy.io.wavfile import write
import pandas as pd
import numpy as np
import os
import sys
import wave, audioop
import IPython as ip
from scipy.io.wavfile import write
from IPython.display import Audio
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt
from gtts import gTTS
import googletrans


In [5]:
# load breast cancer dataset, a well-known small dataset that comes with scikit-learn
from sklearn.datasets import load_breast_cancer
from sklearn import svm
from sklearn.model_selection import train_test_split
breast_cancer_data = load_breast_cancer()
classes = breast_cancer_data.target_names.tolist()

# split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(breast_cancer_data.data,            
                                                    breast_cancer_data.target,  
                                                    test_size=0.2,
                                                    random_state=0)
clf = svm.SVC(gamma=0.001, C=100., probability=True)
model = clf.fit(x_train, y_train)

In [9]:
breast_cancer_data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [6]:

from interpret.ext.blackbox import TabularExplainer

# you can use one of the following four interpretable models as a global surrogate to the black box model

from interpret.ext.glassbox import LGBMExplainableModel
from interpret.ext.glassbox import LinearExplainableModel
from interpret.ext.glassbox import SGDExplainableModel
from interpret.ext.glassbox import DecisionTreeExplainableModel

# "features" and "classes" fields are optional
explainer = TabularExplainer(model, 
                             x_train, 
                             features=breast_cancer_data.feature_names, 
                             classes=classes)

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning


In [9]:

from interpret.ext.blackbox import MimicExplainer

# you can use one of the following four interpretable models as a global surrogate to the black box model

from interpret.ext.glassbox import LGBMExplainableModel
from interpret.ext.glassbox import LinearExplainableModel
from interpret.ext.glassbox import SGDExplainableModel
from interpret.ext.glassbox import DecisionTreeExplainableModel

# "features" and "classes" fields are optional
# augment_data is optional and if true, oversamples the initialization examples to improve surrogate model accuracy to fit original model.  Useful for high-dimensional data where the number of rows is less than the number of columns.
# max_num_of_augmentations is optional and defines max number of times we can increase the input data size.
# LGBMExplainableModel can be replaced with LinearExplainableModel, SGDExplainableModel, or DecisionTreeExplainableModel
explainer = MimicExplainer(model, 
                           x_train, 
                           LGBMExplainableModel, 
                           augment_data=True, 
                           max_num_of_augmentations=10, 
                           features=breast_cancer_data.feature_names, 
                           classes=classes)

In [27]:
pip install translate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
def quiter(filepath):
  factor = 0.01
  with wave.open('%s'%(filepath), 'rb') as wav:
    p = wav.getparams()
    with wave.open('gdrive/My Drive/quiter.wav', 'wb') as audio:
        audio.setparams(p)
        frames = wav.readframes(p.nframes)
        audio.writeframesraw(audioop.mul(frames, p.sampwidth, factor))

def play_audio(y1=None,sample_rate1=None,filepath=None):
  if filepath != None:
    y1, sample_rate1 = librosa.load('%s'%(filepath), mono=True)
  return ip.display.Audio((y1), rate=int((sample_rate1)))


def quieter(filepath):
  y1, sample_rate1 = librosa.load('%s'%(filepath), mono=True)
  return abs(y1-5), sample_rate1

def apply_noise(filename_audio,filename_noise):
  # data, sample_rate = librosa.load(path, duration=2, offset=0.6, sr=8025)
  y1, sample_rate1 = librosa.load('%s'%(filename_audio),duration=2, offset=0.6, sr=8025)
  y2, sample_rate2 = librosa.load('%s'%(filename_noise), mono=True)
  y2 = y2[0:len(y1)]
  return y1+0.025*y2, (sample_rate1+sample_rate2)/2


from translate import Translator
# you can use the training data or the test data here, but test data would allow you to use Explanation Exploration
global_explanation = explainer.explain_local(x_test)

# if you used the PFIExplainer in the previous step, use the next line of code instead
# global_explanation = explainer.explain_global(x_train, true_labels=y_train)

ranked_local_names = global_explanation.get_ranked_local_names()[0][0]

ranked_local_values = global_explanation.get_ranked_local_values()[0][0]

# Create a text explanation
text_explanation = "Feature importance for the first instance:\n"
for i in range(len(ranked_local_names[:5])):
    text_explanation += f"{i+1}. Feature '{ranked_local_names[i]}' has importance {round(ranked_local_values[i], 2)}\n"
def trans():
  translator = Translator(to_lang='es')
  english_text = text_explanation
  translated_text = translator.translate(english_text)

  print(english_text)

  language = 'es'

  myobj = gTTS(text=translated_text, lang=language, slow=False)

  myobj.save("welcome.mp3")
  y,sample_rate = quieter('welcome.mp3')
  return play_audio(y,sample_rate)

In [37]:
trans()

Feature importance for the first instance:
1. Feature 'worst concave points' has importance 2.19
2. Feature 'mean concave points' has importance 1.42
3. Feature 'worst perimeter' has importance 0.55
4. Feature 'worst texture' has importance 0.4
5. Feature 'worst area' has importance 0.35



Feature importance for the first instance


AttributeError: ignored

In [17]:
from gtts import gTTS
from googletrans import Translator

# Translate the English text to Spanish
translator = Translator()
translated_text = translator.translate(english_text)

print(translated_text)

language = 'es'

myobj = gTTS(text=translated_text, lang=language, slow=False)
myobj.save("welcome.mp3")

# Rest of the code for playing the audio file


AttributeError: ignored